In [2]:
from openbb_terminal.sdk import openbb
from financialmodelingprep import fmp_stocks
import yaml
import pandas as pd
from datetime import datetime
from db.schema_models import LogAvailableTickers, LogIngestedTickers
from db.db_utils import insert_data

from db.db_engine import engine


with open('credentials.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Fetch the companies and tickers
openbb.keys.fmp(key=config['api']['fmp'], persist=True)
companies = fmp_stocks.available_companies(config['api']['fmp'])
companies_nasdaq = companies[(companies['exchangeShortName'] == "NASDAQ") & (companies['type'] == "stock")]
list_ticker = companies_nasdaq.symbol.to_list()

In [7]:
from sqlalchemy import inspect

# Insert the data into the database
record = {
    'timestamp': datetime.now(),
    'num_companies': len(companies),
    'num_tickers': len(list_ticker)
}
insert_data(LogAvailableTickers, record)


In [8]:
list_ticker = ['AAPL', "PLTR","AMZN","META","NIO","RGTI","FFIEW"]

In [9]:
source="FinancialModelingPrep"
quarterly=True
limit=100


inspector = inspect(engine)
if 'fa_balance' in inspector.get_table_names():
    df = pd.read_sql('fa_balance', con=engine)
else:
    df = pd.DataFrame()



In [10]:




for c,ticker in enumerate(list_ticker):
    print(c, ticker)
    df_quarterly = openbb.stocks.fa.balance(symbol=ticker, source=source, quarterly=quarterly, limit=limit)
    df_quarterly = df_quarterly.T.reset_index()

    df_quarterly['ticker'] = ticker
    
    # Aggiungiamo la data corrente
    df_quarterly['current_date'] = pd.Timestamp.now().floor('S')
    
    df_quarterly.columns = [col.lower().replace(" ","_") for col in df_quarterly.columns]

    # Prendiamo solo i record che non sono già presenti rispetto a Cik, Calendar year, Period
    
    if df.empty:
        df = df_quarterly.copy()
        
        record = {
            'ingestion_timestamp': datetime.now(),
            'ticker': ticker,
            'num_new_rows': len(df),
            'num_old_rows_diff_dates': 0,
            'num_columns': len(df.columns)
        }
        
    else:
        df_quarterly_merge = df_quarterly.merge(df, how='left', indicator=True, on=['cik', 'calendar_year', 'period'], suffixes=('', '_y'))
        df_new_rows = df_quarterly_merge[df_quarterly_merge['_merge'] == 'left_only']
        
        df = df.append(df_new_rows[df.columns])
            
        # Verifichiamo se ci sono record che sono già presenti ma con date diverse
        df_old_rows = df_quarterly_merge[df_quarterly_merge['_merge'] == 'both']
        
        df_old_rows_diff_dates = df_old_rows[(df_old_rows['filling_date'] != df_old_rows['filling_date_y']) | (df_old_rows['accepted_date'] != df_old_rows['accepted_date_y'])]
        df = df.append(df_old_rows_diff_dates[df.columns])


    # Creiamo e inseriamo il nuovo record in log_ingested_tickers
        record = {
            'ingestion_timestamp': datetime.now(),
            'ticker': ticker,
            'num_new_rows': len(df_new_rows),
            'num_old_rows_diff_dates': len(df_old_rows_diff_dates),
            'num_columns': len(df_quarterly.columns)
        }
        
    insert_data(LogIngestedTickers, record)
    
    df.to_sql('fa_balance', engine, if_exists='replace', index=False)


0 AAPL
1 PLTR
2 AMZN
3 META
4 NIO
5 RGTI
6 FFIEW
